In [35]:
import os
import json
import re

In [ ]:
# def ConvertVocab(path):
#     index = 0
#     vocab = dict()
#     with open(path, "r", encoding="utf-8") as f:
#         for word in f:            
#             vocab[word.strip()] = index
#             index += 1
#     with open("vocabBert.txt", "w") as f:
#         json.dump(vocab, f)

In [46]:
class tokenizer:
    def __init__(self, path = r"C:\Users\shiva\Desktop\IISC\code\NeuroCpp\Projects\The Dream\embedding\vocabBert.txt"):
        self.path = path
        with open(path, "r") as f:
            self.vocab = json.load(f)
            
        self.reverseVocab = dict()
        for i in self.vocab.keys():
            self.reverseVocab[self.vocab[i]] = i    

    def EncodeWord(self, word):
        if(word in self.vocab):
            return [self.vocab[word]]
        res = []
        for i in range(len(word) - 1, -1, -1):
            if(word[:i] in self.vocab):
                res.append(self.vocab[word[:i]])
                res.extend(self.EncodeWord("##" + word[i:]))
                break
        return res
    
    def SplitWord(self, txt):
        res = []
        currentTxt = ""
        for ch in txt:
            if(ch == " "):
                if(currentTxt != ""):
                    res.append(currentTxt)
                currentTxt = ""
                continue
            elif(re.match(r"[^a-z0-9]", ch)):
                if(currentTxt != ""):
                    res.append(currentTxt)
                res.append(ch)
                currentTxt = ""
                continue
            currentTxt += ch
        return res
                

    def encode(self, txt):
        txt = txt.replace("\n", " ").lower()
        words = self.SplitWord(txt)
        res = []
        for word in words:
            res.extend(self.EncodeWord(word))
        return res
    
    def decode(self, li):
        pass

In [51]:
tkn = tokenizer()
tkn.encode("  Hello,,,   WORLD!! 42@@@openAI##   is---great??? \n\n   NLP---rocks :)    [unused10] ".lower())

[7592,
 1010,
 1010,
 1010,
 2088,
 999,
 999,
 4413,
 1030,
 1030,
 1030,
 2330,
 4886,
 1001,
 1001,
 2003,
 1011,
 1011,
 1011,
 2307,
 1029,
 1029,
 1029,
 17953,
 2361,
 1011,
 1011,
 1011,
 5749,
 1024,
 1007,
 1031,
 15171,
 10790,
 1033]